In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
par = pd.read_parquet('DATA/speaker_attributes.parquet', engine='pyarrow')

C:\Users\lucaw\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [ ]:
wikidata = pd.read_csv('DATA/wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')

In [3]:
par

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055976,[Barker Howard],None,[Q30],[Q6581097],1397399351,None,None,[Q82955],None,None,Q106406560,Barker B. Howard,None,item,None
9055977,[Charles Macomber],None,[Q30],[Q6581097],1397399471,None,None,[Q82955],None,None,Q106406571,Charles H. Macomber,None,item,None
9055978,None,[+1848-04-01T00:00:00Z],None,[Q6581072],1397399751,None,None,None,None,None,Q106406588,Dina David,None,item,None
9055979,None,[+1899-03-18T00:00:00Z],None,[Q6581072],1397399799,None,None,None,None,None,Q106406593,Irma Dexinger,None,item,None


### Parse attributes ([QID] -> [values])

In [ ]:
f = lambda x: x[0]
gend = par['gender'].fillna('inc').values
res = wikidata.loc[np.array([f(x) for x in gend])]['Label'].values
par['gender_parse'] = res

In [102]:
g = lambda x: [wikidata.loc[x[i]]['Label'] for i in range(len(x))]
nationality = par['nationality'].dropna()
nationality_parse = nationality.apply(g)
nat_parse = par['nationality'].copy()
nat_parse[par['nationality'].notnull()] = nationality_parse.copy()
par['nationality_parse'] = nat_parse.copy()

In [105]:
religion = par['religion'].dropna()
religion_parse = religion.apply(g)
rel_parse = par['religion'].copy()
rel_parse[par['religion'].notnull()] = religion_parse.copy()
par['religion_parse'] = rel_parse.copy()

In [113]:
candidacy = par['candidacy'].dropna()
candidacy_parse = candidacy.apply(g)
cand_parse = par['candidacy'].copy()
cand_parse[par['candidacy'].notnull()] = candidacy_parse.copy()
par['candidacy_parse'] = cand_parse.copy()

In [116]:
ethnic_group = par['ethnic_group'].dropna()
ethnic_group_parse = ethnic_group.apply(g)
eth_parse = par['ethnic_group'].copy()
eth_parse[par['ethnic_group'].notnull()] = ethnic_group_parse.copy()
par['ethnic_group_parse'] = eth_parse.copy()

In [127]:
g = lambda x: [wikidata.loc[x[i]]['Label'] if x[i] in wikidata.index else None for i in range(len(x))]
occupation = par['occupation'].dropna()
occupation_parse = occupation.apply(g)
occ_parse = par['occupation'].copy()
occ_parse[par['occupation'].notnull()] = occupation_parse.copy()
par['occupation_parse'] = occ_parse.copy()

In [129]:
party = par['party'].dropna()
party_parse = party.apply(g)
part_parse = par['party'].copy()
part_parse[par['party'].notnull()] = party_parse.copy()
par['party_parse'] = part_parse.copy()

In [131]:
academic_degree = par['academic_degree'].dropna()
academic_degree_parse = academic_degree.apply(g)
acc_parse = par['academic_degree'].copy()
acc_parse[par['academic_degree'].notnull()] = academic_degree_parse.copy()
par['academic_degree_parse'] = acc_parse.copy()

In [138]:
speaker_attributes_parse = par[['aliases', 'date_of_birth', 'nationality_parse', 'gender_parse', 'ethnic_group_parse',
                               'US_congress_bio_ID', 'occupation_parse', 'party_parse', 'academic_degree_parse', 'label', 
                               'candidacy_parse', 'religion_parse']]

In [140]:
speaker_attributes_parse = speaker_attributes_parse.rename(columns={'nationality_parse': 'nationality',
                                                                    'gender_parse': 'gender',
                                                                    'ethnic_group_parse': 'ethnic_group',
                                                                    'occupation_parse': 'occupation',
                                                                    'party_parse': 'party',
                                                                    'academic_degree_parse': 'academic_degree',
                                                                    'candidacy_parse': 'candidacy',
                                                                    'religion_parse': 'religion'})
speaker_attributes_parse

,aliases,date_of_birth,nationality,gender,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,label,candidacy,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Great Britain, United States of America]",male,None,W000178,"[politician, military officer, farmer, cartogr...",[independent politician],None,George Washington,"[1792 United States presidential election, 178...",[Episcopal Church]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[United Kingdom],male,[White British],None,"[playwright, screenwriter, novelist, children'...",None,None,Douglas Adams,None,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Belgium],male,None,None,"[writer, lawyer, librarian, information scient...",None,None,Paul Otlet,None,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[United States of America],male,None,None,"[politician, motivational speaker, autobiograp...",[Republican Party],None,George W. Bush,"[2000 United States presidential election, 200...","[United Methodist Church, Episcopal Church, Me..."
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Spain],male,None,None,[painter],None,None,Diego Velázquez,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
9055976,[Barker Howard],None,[United States of America],male,None,None,[politician],None,None,Barker B. Howard,None,None
9055977,[Charles Macomber],None,[United States of America],male,None,None,[politician],None,None,Charles H. Macomber,None,None
9055978,None,[+1848-04-01T00:00:00Z],None,female,None,None,None,None,None,Dina David,None,None
9055979,None,[+1899-03-18T00:00:00Z],None,female,None,None,None,None,None,Irma Dexinger,None,None


In [141]:
speaker_attributes_parse.to_json('DATA/speaker_attributes_parse.json')